In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = '/content/drive/MyDrive/braintumor_dataset'  # Update with the actual path to your dataset
batch_size = 32 # Updated batch size to 32
epochs = 10
validation_split = 0.1
input_shape = (224, 224, 3)  # Adjust based on your dataset's image dimensions
num_classes = 2  # Two classes: 'no-tumor' and 'meningioma-tumor'

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=validation_split,
)

In [ ]:
train_generator = datagen.flow_from_directory(
    data_dir + '/Training',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
)

Found 3971 images belonging to 2 classes.


In [ ]:
validation_generator = datagen.flow_from_directory(
    data_dir + '/Training',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

Found 440 images belonging to 2 classes.


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
)

Epoch 1/10
124/124 [==============================] - 350s 3s/step - loss: 0.1099 - accuracy: 0.9554 - val_loss: 0.5548 - val_accuracy: 0.8365
Epoch 2/10
124/124 [==============================] - 350s 3s/step - loss: 0.1156 - accuracy: 0.9597 - val_loss: 0.5306 - val_accuracy: 0.8077
Epoch 3/10
124/124 [==============================] - 355s 3s/step - loss: 0.1067 - accuracy: 0.9657 - val_loss: 0.5462 - val_accuracy: 0.7885
Epoch 4/10
124/124 [==============================] - 353s 3s/step - loss: 0.1004 - accuracy: 0.9594 - val_loss: 0.4255 - val_accuracy: 0.8606
Epoch 5/10
124/124 [==============================] - 357s 3s/step - loss: 0.1004 - accuracy: 0.9672 - val_loss: 0.7268 - val_accuracy: 0.7885
Epoch 6/10
124/124 [==============================] - 346s 3s/step - loss: 0.1042 - accuracy: 0.9609 - val_loss: 0.4844 - val_accuracy: 0.8173
Epoch 7/10
124/124 [==============================] - 353s 3s/step - loss: 0.0921 - accuracy: 0.9635 - val_loss: 0.3260 - val_accuracy: 0.8654

In [ ]:
test_generator = datagen.flow_from_directory(
    data_dir + '/Testing',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
)

Found 1011 images belonging to 2 classes.


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

32/32 [==============================] - 215s 7s/step - loss: 0.3259 - accuracy: 0.9021
Test Loss: 0.3259
Test Accuracy: 90.21%


In [ ]:
model.save('inceptionv3braintumor.h5')